In [1]:
import numpy as np
from __future__ import print_function
from keras.utils import plot_model

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np


def decoder_input_txt(encode_input):
    shape1=np.shape(encode_input)
    len1=shape1[1]
    decode=''
    for i in range (len1):
        index = np.argmax(in1[0, i, :])
        ch1=reverse_input_char_index[index]
        decode=decode+ch1
    return decode

def decoder_target_txt(encode_input):
    shape1=np.shape(encode_input)
    len1=shape1[1]
    decode=''
    for i in range (len1):
        index = np.argmax(encode_input[0, i, :])
        ch1=reverse_target_char_index[index]
        decode=decode+ch1
    return decode


batch_size = 64  # Batch size for training.
epochs = 1000  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 50  # Number of samples to train on.
# Path to the data txt file on disk.
#text = 'C:/Users/BRG_user/Desktop/nl_only.txt'
#sparql='C:/Users/BRG_user/Desktop/sparql_only.txt'

text = 'C:/Users/BRG_user/Desktop/small_vocab_fr'
sparql='C:/Users/BRG_user/Desktop/small_vocab_en'



C:\Users\BRG_user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:


# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
i=0
with open(text, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text = line
    #print(line)
    i=i+1
    #print('------')
    input_texts.append(input_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
print(i)            
i=0
            
with open(sparql, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    target_text = line 
    #print(line)
    #print('------')
    i=i+1
    target_text = '\t' + target_text + '\n'
    target_texts.append(target_text)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
print(i)  

50
50


In [3]:
# input_characters = sorted list of unique character in input
# target_characters = sorted list of unique character in output
# num_encoder_tokens = the number of unique character in input
# num_decoder_tokens = the number of unique character in output
# max_encoder_seq_length = number of character in the longest string of input
# max_decoder_seq_length = number of character in the longest string of output

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('----------------------------------------------------------------------------------------------------------------------------')
print('Number of unique input tokens (unique character):', num_encoder_tokens)
print('----------------------------------------------------------------------------------------------------------------------------')
print('Number of unique output tokens (unique character) :', num_decoder_tokens)
print('----------------------------------------------------------------------------------------------------------------------------')
print('Max sequence length for inputs (longest sentence len) :', max_encoder_seq_length)
print('----------------------------------------------------------------------------------------------------------------------------')
print('Max sequence length for outputs (longest sentence len):', max_decoder_seq_length)
print('----------------------------------------------------------------------------------------------------------------------------')
print('input_characters :   ', input_characters)
print('----------------------------------------------------------------------------------------------------------------------------')
print('target_characters :  ', target_characters)

Number of samples: 50
----------------------------------------------------------------------------------------------------------------------------
Number of unique input tokens (unique character): 34
----------------------------------------------------------------------------------------------------------------------------
Number of unique output tokens (unique character) : 31
----------------------------------------------------------------------------------------------------------------------------
Max sequence length for inputs (longest sentence len) : 90
----------------------------------------------------------------------------------------------------------------------------
Max sequence length for outputs (longest sentence len): 93
----------------------------------------------------------------------------------------------------------------------------
input_characters :    [' ', "'", ',', '-', '.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q',

In [4]:
#input_token_index is a dictionary . we want to create index for each unique character from input
#target_token_index  is a dictionary . we want to create index for each unique character from output
# num_encoder_tokens = the number of unique character in input
# num_decoder_tokens = the number of unique character in output
# max_encoder_seq_length = number of character in the longest string of input
# max_decoder_seq_length = number of character in the longest string of output
# input_characters = sorted list of unique character in input
# target_characters = sorted list of unique character in output

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# as a sample in input, we have 50 sentences and the longest one has 90 characters  and we have 34 unique character 
# so we can show 50 sentences with  (50, 90, 34) matrix

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')

# as a sample we have 50 sentences in output, and the longest one has 93 characters  and we have 31 unique character 
# so we can show decode 50 output sentences with  (50, 93, 31) matrix

decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

# as a sample in output, we have 50 sentences and the longest one has 93 characters  and we have 31 unique character 
# so we can show 50 output sentences with  (50, 93, 31) matrix
#The only differences between decoder_input_data and decoder_target_data is that we shift decoder_input_data by 1 and put it
# in   decoder_target_data  so decoder_input_data[i,j,:] =decoder_target_data[i,j-1,:] 

decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

print('create index for input unique character : ',input_token_index)

print('----------------------------------------------------------------------------------------------------------------------------')

print('create index for output unique character : ',target_token_index)

print('----------------------------------------------------------------------------------------------------------------------------')

print('encoder_input_data shape ',np.shape(encoder_input_data))

print('----------------------------------------------------------------------------------------------------------------------------')

print('decoder_input_data shape ',np.shape(decoder_input_data))

print('----------------------------------------------------------------------------------------------------------------------------')

print('decoder_target_data shape ',np.shape(decoder_target_data))


create index for input unique character :  {'ê': 32, 'l': 15, 'è': 30, "'": 1, 'c': 7, 'p': 19, 'y': 28, 'g': 11, ' ': 0, 'é': 31, 'v': 25, 'w': 26, 'q': 20, 'n': 17, 'à': 29, 'b': 6, 'u': 24, 'h': 12, '-': 3, 'a': 5, '.': 4, 'j': 14, 'r': 21, 'e': 9, 'f': 10, 'x': 27, 't': 23, 'd': 8, ',': 2, 'm': 16, 's': 22, 'i': 13, 'o': 18, 'û': 33}
----------------------------------------------------------------------------------------------------------------------------
create index for output unique character :  {'l': 16, 'b': 6, 'z': 30, 'c': 7, 'y': 29, 'g': 11, ' ': 2, 'v': 26, 'w': 27, 'q': 21, 'n': 18, 'p': 20, 's': 23, 'u': 25, 'h': 12, 'a': 5, '.': 4, 'j': 14, 'r': 22, 'e': 9, 'f': 10, 'x': 28, 't': 24, 'd': 8, '\n': 1, ',': 3, 'm': 17, '\t': 0, 'i': 13, 'o': 19, 'k': 15}
----------------------------------------------------------------------------------------------------------------------------
encoder_input_data shape  (50, 90, 34)
-------------------------------------------------------

In [5]:
# encoder_input_data shape  (50, 90, 34)
# ----------------------------------------------------------------------------------------------------------------------------
# decoder_input_data shape  (50, 93, 31)
# ----------------------------------------------------------------------------------------------------------------------------
# decoder_target_data shape  (50, 93, 31)

#input_token_index is a dictionary . we want to create index for each unique character from input
#target_token_index  is a dictionary . we want to create index for each unique character from output
# num_encoder_tokens = the number of unique character in input
# num_decoder_tokens = the number of unique character in output
# max_encoder_seq_length = number of character in the longest string of input
# max_decoder_seq_length = number of character in the longest string of output
# input_characters = sorted list of unique character in input
# target_characters = sorted list of unique character in output


for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
        
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


In [6]:
print('The first character of the first sentence  in input ',encoder_input_data[1,1,:])

print('----------------------------------------------------------------------------------------------------------------------------')

print('The first character of the first sentence  in output',decoder_input_data[1,1,:])

print('----------------------------------------------------------------------------------------------------------------------------')

print('The second  character of the first sentence  in output ',decoder_input_data[1,2,:])

print('----------------------------------------------------------------------------------------------------------------------------')


print('the only differences between decoder_input_data and decoder_target_data is that we shift decoder_input_data by 1 and put it')
print('in   decoder_target_data  so decoder_input_data[i,j,:] =decoder_target_data[i,j-1,:]  \n  ',decoder_target_data[1,1,:])

The first character of the first sentence  in input  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
----------------------------------------------------------------------------------------------------------------------------
The first character of the first sentence  in output [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0.]
----------------------------------------------------------------------------------------------------------------------------
The second  character of the first sentence  in output  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
----------------------------------------------------------------------------------------------------------------------------
the only differences between decoder_input_data and decoder_target_data is that we shift decoder_input_data by 1 and put it
in   decoder_target_data  so decoder_input

In [7]:

# num_encoder_tokens = the number of unique character in input
# num_decoder_tokens = the number of unique character in output
# max_encoder_seq_length = number of character in the longest string of input
# max_decoder_seq_length = number of character in the longest string of output
# input_characters = sorted list of unique character in input
# target_characters = sorted list of unique character in output

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))  # num_encoder_tokens = the number of unique character in input

# latent_dim = 256  --- Latent dimensionality of the encoding space.

encoder = LSTM(latent_dim, return_state=True)

encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens)) # num_decoder_tokens = the number of unique character in output
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)


decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


In [8]:

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')



Train on 40 samples, validate on 10 samples
Epoch 1/1000
40/40 [==============================] - 3s 72ms/step - loss: 2.4936 - val_loss: 2.6591
Epoch 2/1000
40/40 [==============================] - 0s 8ms/step - loss: 2.4560 - val_loss: 2.5077
Epoch 3/1000
40/40 [==============================] - 0s 9ms/step - loss: 2.3282 - val_loss: 3.2944
Epoch 4/1000
40/40 [==============================] - 0s 10ms/step - loss: 3.0417 - val_loss: 2.5055
Epoch 5/1000
40/40 [==============================] - 0s 12ms/step - loss: 2.3296 - val_loss: 2.3027
Epoch 6/1000
40/40 [==============================] - 1s 13ms/step - loss: 2.1378 - val_loss: 2.2752
Epoch 7/1000
40/40 [==============================] - 1s 13ms/step - loss: 2.1055 - val_loss: 2.2650
Epoch 8/1000
40/40 [==============================] - 1s 15ms/step - loss: 2.0953 - val_loss: 2.2577
Epoch 9/1000
40/40 [==============================] - 0s 12ms/step - loss: 2.0888 - val_loss: 2.2522
Epoch 10/1000
40/40 [============================

40/40 [==============================] - 0s 11ms/step - loss: 1.7715 - val_loss: 1.9408
Epoch 82/1000
40/40 [==============================] - 0s 12ms/step - loss: 1.7999 - val_loss: 2.0184
Epoch 83/1000
40/40 [==============================] - 0s 12ms/step - loss: 1.8429 - val_loss: 1.9333
Epoch 84/1000
40/40 [==============================] - 0s 11ms/step - loss: 1.8324 - val_loss: 1.9331
Epoch 85/1000
40/40 [==============================] - 0s 11ms/step - loss: 1.7863 - val_loss: 1.9098
Epoch 86/1000
40/40 [==============================] - 0s 12ms/step - loss: 1.7858 - val_loss: 1.9078
Epoch 87/1000
40/40 [==============================] - 0s 11ms/step - loss: 1.7465 - val_loss: 1.8815
Epoch 88/1000
40/40 [==============================] - 0s 11ms/step - loss: 1.7540 - val_loss: 1.9159
Epoch 89/1000
40/40 [==============================] - 0s 11ms/step - loss: 1.7660 - val_loss: 1.8645
Epoch 90/1000
40/40 [==============================] - 0s 9ms/step - loss: 1.7420 - val_loss: 1.

40/40 [==============================] - 0s 12ms/step - loss: 1.1768 - val_loss: 1.3525
Epoch 162/1000
40/40 [==============================] - 1s 13ms/step - loss: 1.1619 - val_loss: 1.3030
Epoch 163/1000
40/40 [==============================] - 0s 12ms/step - loss: 1.1509 - val_loss: 1.3090
Epoch 164/1000
40/40 [==============================] - 0s 11ms/step - loss: 1.1259 - val_loss: 1.3378
Epoch 165/1000
40/40 [==============================] - 1s 13ms/step - loss: 1.1400 - val_loss: 1.2627
Epoch 166/1000
40/40 [==============================] - 1s 13ms/step - loss: 1.1130 - val_loss: 1.2938
Epoch 167/1000
40/40 [==============================] - 1s 13ms/step - loss: 1.1169 - val_loss: 1.3217
Epoch 168/1000
40/40 [==============================] - 0s 12ms/step - loss: 1.1271 - val_loss: 1.2484
Epoch 169/1000
40/40 [==============================] - 0s 10ms/step - loss: 1.0987 - val_loss: 1.3085
Epoch 170/1000
40/40 [==============================] - 1s 13ms/step - loss: 1.0985 - va

Epoch 241/1000
40/40 [==============================] - 0s 11ms/step - loss: 0.6943 - val_loss: 0.9922
Epoch 242/1000
40/40 [==============================] - 0s 10ms/step - loss: 0.7011 - val_loss: 1.0315
Epoch 243/1000
40/40 [==============================] - 0s 9ms/step - loss: 0.6958 - val_loss: 0.9900
Epoch 244/1000
40/40 [==============================] - 1s 13ms/step - loss: 0.7197 - val_loss: 1.0199
Epoch 245/1000
40/40 [==============================] - 0s 12ms/step - loss: 0.6723 - val_loss: 0.9893
Epoch 246/1000
40/40 [==============================] - 0s 12ms/step - loss: 0.6937 - val_loss: 0.9955
Epoch 247/1000
40/40 [==============================] - 1s 14ms/step - loss: 0.6834 - val_loss: 1.0496
Epoch 248/1000
40/40 [==============================] - 1s 15ms/step - loss: 0.7019 - val_loss: 0.9623
Epoch 249/1000
40/40 [==============================] - 0s 12ms/step - loss: 0.7120 - val_loss: 1.0130
Epoch 250/1000
40/40 [==============================] - 1s 13ms/step - los

Epoch 321/1000
40/40 [==============================] - 0s 11ms/step - loss: 0.4024 - val_loss: 0.8882
Epoch 322/1000
40/40 [==============================] - 0s 11ms/step - loss: 0.3588 - val_loss: 0.8786
Epoch 323/1000
40/40 [==============================] - 0s 12ms/step - loss: 0.3478 - val_loss: 0.8628
Epoch 324/1000
40/40 [==============================] - 0s 11ms/step - loss: 0.3367 - val_loss: 0.8959
Epoch 325/1000
40/40 [==============================] - 1s 13ms/step - loss: 0.3293 - val_loss: 0.8400
Epoch 326/1000
40/40 [==============================] - 0s 10ms/step - loss: 0.3280 - val_loss: 0.9314
Epoch 327/1000
40/40 [==============================] - 0s 8ms/step - loss: 0.3353 - val_loss: 0.8434
Epoch 328/1000
40/40 [==============================] - 0s 8ms/step - loss: 0.3618 - val_loss: 0.9348
Epoch 329/1000
40/40 [==============================] - 0s 8ms/step - loss: 0.3584 - val_loss: 0.9039
Epoch 330/1000
40/40 [==============================] - 1s 13ms/step - loss:

...

Epoch 721/1000
40/40 [==============================] - 0s 11ms/step - loss: 0.0511 - val_loss: 1.1067
Epoch 722/1000
40/40 [==============================] - 1s 13ms/step - loss: 0.0509 - val_loss: 1.1085
Epoch 723/1000
40/40 [==============================] - 0s 12ms/step - loss: 0.0506 - val_loss: 1.1103
Epoch 724/1000
40/40 [==============================] - 1s 13ms/step - loss: 0.0503 - val_loss: 1.1112
Epoch 725/1000
40/40 [==============================] - 1s 13ms/step - loss: 0.0500 - val_loss: 1.1147
Epoch 726/1000
40/40 [==============================] - 0s 12ms/step - loss: 0.0498 - val_loss: 1.1118
Epoch 727/1000
40/40 [==============================] - 0s 12ms/step - loss: 0.0500 - val_loss: 1.1245
Epoch 728/1000
40/40 [==============================] - 1s 14ms/step - loss: 0.0521 - val_loss: 1.1145
Epoch 729/1000
40/40 [==============================] - 1s 13ms/step - loss: 0.0617 - val_loss: 1.1343
Epoch 730/1000
40/40 [==============================] - 1s 13ms/step - lo

Epoch 801/1000
40/40 [==============================] - 0s 11ms/step - loss: 0.0607 - val_loss: 1.0768
Epoch 802/1000
40/40 [==============================] - 1s 13ms/step - loss: 0.1016 - val_loss: 1.1514
Epoch 803/1000
40/40 [==============================] - 1s 14ms/step - loss: 0.1260 - val_loss: 1.1166
Epoch 804/1000
40/40 [==============================] - 1s 13ms/step - loss: 0.0771 - val_loss: 1.0975
Epoch 805/1000
40/40 [==============================] - 1s 14ms/step - loss: 0.0435 - val_loss: 1.0739
Epoch 806/1000
40/40 [==============================] - 0s 12ms/step - loss: 0.0339 - val_loss: 1.0850
Epoch 807/1000
40/40 [==============================] - 1s 14ms/step - loss: 0.0323 - val_loss: 1.0934
Epoch 808/1000
40/40 [==============================] - 0s 10ms/step - loss: 0.0319 - val_loss: 1.1017
Epoch 809/1000
40/40 [==============================] - 1s 13ms/step - loss: 0.0339 - val_loss: 1.1106
Epoch 810/1000
40/40 [==============================] - 0s 12ms/step - lo

Epoch 881/1000
40/40 [==============================] - 1s 14ms/step - loss: 0.0321 - val_loss: 1.2087
Epoch 882/1000
40/40 [==============================] - 0s 12ms/step - loss: 0.0343 - val_loss: 1.2026
Epoch 883/1000
40/40 [==============================] - 0s 8ms/step - loss: 0.0314 - val_loss: 1.2107
Epoch 884/1000
40/40 [==============================] - 0s 7ms/step - loss: 0.0285 - val_loss: 1.1937
Epoch 885/1000
40/40 [==============================] - 0s 8ms/step - loss: 0.0287 - val_loss: 1.2214
Epoch 886/1000
40/40 [==============================] - 1s 13ms/step - loss: 0.0295 - val_loss: 1.1950
Epoch 887/1000
40/40 [==============================] - 1s 13ms/step - loss: 0.0344 - val_loss: 1.2052
Epoch 888/1000
40/40 [==============================] - 0s 11ms/step - loss: 0.0311 - val_loss: 1.1981
Epoch 889/1000
40/40 [==============================] - 1s 13ms/step - loss: 0.0322 - val_loss: 1.1810
Epoch 890/1000
40/40 [==============================] - 0s 12ms/step - loss:

Epoch 961/1000
40/40 [==============================] - 0s 11ms/step - loss: 0.0475 - val_loss: 1.1789
Epoch 962/1000
40/40 [==============================] - 0s 11ms/step - loss: 0.0869 - val_loss: 1.2494
Epoch 963/1000
40/40 [==============================] - 0s 12ms/step - loss: 0.0671 - val_loss: 1.1609
Epoch 964/1000
40/40 [==============================] - 0s 12ms/step - loss: 0.0379 - val_loss: 1.2148
Epoch 965/1000
40/40 [==============================] - 0s 12ms/step - loss: 0.0264 - val_loss: 1.1917
Epoch 966/1000
40/40 [==============================] - 0s 11ms/step - loss: 0.0223 - val_loss: 1.1972
Epoch 967/1000
40/40 [==============================] - 0s 11ms/step - loss: 0.0221 - val_loss: 1.1896
Epoch 968/1000
40/40 [==============================] - 0s 10ms/step - loss: 0.0195 - val_loss: 1.1973
Epoch 969/1000
40/40 [==============================] - 0s 11ms/step - loss: 0.0184 - val_loss: 1.2021
Epoch 970/1000
40/40 [==============================] - 0s 10ms/step - lo

C:\Users\BRG_user\Anaconda3\lib\site-packages\keras\engine\network.py:888: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [9]:
from keras.utils import plot_model

#plot_model(model, to_file='model.png')
model.summary()

# inputs1 = Input(shape=(3, 1)) # batch shape = none
# lstm1 = LSTM(1)(inputs1) 
# print(decoder_lstm.shape)



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 34)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 31)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 297984      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  294912      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [10]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states



# encoder_inputs = Input(shape=(None, num_encoder_tokens))  # num_encoder_tokens = the number of unique character in input

# encoder = LSTM(latent_dim, return_state=True) # latent_dim = 256  

# encoder_outputs, state_h, state_c = encoder(encoder_inputs)




# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 34)          0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 256), (None, 256) 297984    
Total params: 297,984
Trainable params: 297,984
Non-trainable params: 0
_________________________________________________________________


In [11]:





decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


# # Set up the decoder, using `encoder_states` as initial state.
# decoder_inputs = Input(shape=(None, num_decoder_tokens)) # num_decoder_tokens = the number of unique character in output
# # We set up our decoder to return full output sequences,
# # and to return internal states as well. We don't use the
# # return states in the training model, but we will use them in inference.
# decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
# decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)

#decoder_inputs = Input(shape=(None, num_decoder_tokens)) # num_decoder_tokens = the number of unique character in output
#decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]

decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)


decoder_model.summary()
# Reverse-lookup token index to decode sequences back to
# something readable.

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, 31)     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  294912      input_2[0][0]                    
                                                                 input_3[0][0]                    
          

In [12]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [13]:

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence



In [14]:

for seq_index in range(400):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
Decoded sentence: the grapefruit is my most loved fruit , but the banana is her most loved .

-
Input sentence: les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .
Decoded sentence: the united states is never wet during january , but it is usually hot in october .

-
Input sentence: california est généralement calme en mars , et il est généralement chaud en juin .
Decoded sentence: california is usually freezing during december , and it is busy in april .

-
Input sentence: les états-unis est parfois légère en juin , et il fait froid en septembre .
Decoded sentence: the united states is sometimes mild during june , and it is cold in september .

-
Input sentence: votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .
Decoded sentence: the grapefruit is my most loved fruit , but the banana is her most loved .

-
Input sentence: son fr

-
Input sentence: france est merveilleux au mois de novembre , mais il est parfois chaud en septembre .
Decoded sentence: china is usually pleasant during november , and it is never quiet in october .

-
Input sentence: l' inde est généralement agréable en novembre , mais il est jamais relaxant en juillet .
Decoded sentence: the united states is sometimes busy during january , and it is sometimes warm in november .

-
Input sentence: les états-unis ne sont jamais gel pendant l' automne , mais il est jamais occupé en juin .
Decoded sentence: the united states is never nice during february , but it is sometimes pleasant in april .

-
Input sentence: paris est parfois chaud en juin , mais il est généralement chaud en juillet .
Decoded sentence: paris is never pleasant during september , and it is beautiful in autumn .



ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 3 array(s), but instead got the following list of 1 arrays: [array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])]...